# Đồ án môn Lập trình song song ứng dụng
**Giảng viên: Thầy Trần Trung Kiên**

## Thông tin nhóm
### STT: Nhóm 1

### Thành viên:
1. 1712284 - Hoàng Gia Bảo
2. 1712685 - Trần Huy Phượng

## 1. Mô tả ứng dụng

Tên ứng dụng: **Phát hiện cộng đồng trong mạng xã hội bằng thuật toán Girvan Newman**

* Input: Đồ thị vô hướng, không trọng số

* Output: Tập hợp các cộng đồng được phát hiện - mỗi cộng đồng được thể hiện là list các node

* Ý nghĩa thực tế: có rất nhiều, ví dụ một số ứng dụng:
  * Tiếp thị: xác định các cụm khách hàng có chung sở thích trong một mạng thể hiện quan hệ giữa người mua và sản phẩm trên sàn thương mại điện tử để xây dựng hệ thống tư vấn hiệu quả
  * Kết bạn, mở rộng mối quan hệ
  * ...


Khái niệm cộng đồng, được định nghĩa là tập các thực thể có những tính chất 
tương tự nhau hoặc cùng đóng một vai trò trong một mạng xã hội. 

Hình dưới đây thể hiện ví dụ về output của bài toán

In [1]:
%%html
<img src='https://drive.google.com/uc?export=view&amp;id=1COUG0WrM545iqLHDcBYtjD0PkYNc5arQ'/>

Ứng dụng này cần phải được tăng tốc do vì khi recommend 1 sản phẩm cho khách hàng, ta cần thời gian nhanh để đem lại trải nghiệm tốt nhất cho khách hàng. Trong khi đó thuật toán Girvan Newman có thời gian thực thi khá chậm nên việc song song hóa là rất cần thiết.

## 2. Tiền xử lí dữ liệu

### 1. Tải dữ liệu

In [2]:
!wget https://raw.githubusercontent.com/mansiganatra/Girvan-Newman-Implementation-using-Spark/master/ub_sample_data.csv

--2022-05-18 14:40:54--  https://raw.githubusercontent.com/mansiganatra/Girvan-Newman-Implementation-using-Spark/master/ub_sample_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1777828 (1.7M) [text/plain]
Saving to: ‘ub_sample_data.csv.1’

ub_sample_data.csv. 100%[===================>]   1.70M  --.-KB/s    in 0.05s   

2022-05-18 14:40:55 (30.8 MB/s) - ‘ub_sample_data.csv.1’ saved [1777828/1777828]



In [3]:
import csv

In [4]:
fileData = []
header = []
with open('ub_sample_data.csv', newline ='') as csvfile:
    file = csv.reader(csvfile, delimiter=',')
    for row in file:
        if not header:
            header.append(row)
            continue
        fileData.append(row)
    csvfile.close()

### 2. Tạo graph từ input

#### Tạo graph

In [5]:
filterThreshold = 7
setUsers = {}

for [user, item] in fileData:
    if user not in setUsers:
        setUsers[user] = {item}
    else:
        setUsers[user].add(item)

gPyObject = {}

# filter and make adjacency list of graph
for user1, item1 in setUsers.items():
    for user2, item2 in setUsers.items():
        if user1 == user2: continue
        if(len(item1 & item2) >= filterThreshold):
            if user1 not in gPyObject:
                gPyObject[user1] = [user2]
            else:
                gPyObject[user1].append(user2)
cnt = 0
for key, val in gPyObject.items():
    print(key, val)
    cnt += 1
    if cnt == 5:
        break

39FT2Ui8KUXwmUt6hnwy-g ['0FVcoJko1kfZCrJRfssfIA', 'JM0GL6Dx4EuZ1mprLk5Gyg', 'bSUS0YcvS7UelmHvCzNWBA', 'DKolrsBSwMTpTJL22dqJRQ', 'sdLns7062kz3Ur_b8wgeYw', '_VTEyUzzH92X3w-IpGaXVA', 'qtOCfMTrozmUSHWIcohc6Q', 'zBi_JWB5uUdVuz3JLoAxGQ', 'KLB3wIYUwKDPMbijIE92vg', '1KQi8Ymatd4ySAd4fhSfaw', '_Pn-EmWO-pFPFg81ZIEiDw', 'dTeSvET2SR5LDF_J07wJAQ', 'ay4M5J28kBUf0odOQct0BA', 'dzJDCQ5vubQBJTfYTEmcbg', 'mu4XvWvJOb3XpG1C_CHCWA', 'OoyQYSeYNyRVOmdO3tsxYA', 'PE8s8ACYABRNANI-T_WmzA', '79yaBDbLASfIdB-C2c8DzA', 'Uo5dPwoDpYBzOnmUnjxJ6A', 'R4l3ONHzGBakKKNo4TN9iQ', 'bHufZ2OTlC-OUxBDRXxViw', 'ChshgCKJTdIDg17JKtFuJw', 'B0ENvYKQdNNr1Izd2r-BAA']
0FVcoJko1kfZCrJRfssfIA ['39FT2Ui8KUXwmUt6hnwy-g', 'LcCRMIDz1JgshpPGYfLDcA', 'bSUS0YcvS7UelmHvCzNWBA', 'DKolrsBSwMTpTJL22dqJRQ', '_VTEyUzzH92X3w-IpGaXVA', 'qtOCfMTrozmUSHWIcohc6Q', 'zBi_JWB5uUdVuz3JLoAxGQ', 'EiwxlbR8fb68lMgEXhcWKA', '4pc_EyanaC3ARh0MZZyouA', 'KLB3wIYUwKDPMbijIE92vg', '2XYdguaaZ7dgi6fAlddujg', 'bE7Yd0jI_P6g27MWEKKalA', '1KQi8Ymatd4ySAd4fhSfaw', '_Pn-EmWO-pFPFg8

#### Thay đổi định dạng user từ string sang integer

In [6]:
transferUser = {}
cnt = 0
for key in gPyObject.keys():
    transferUser[key] = cnt
    cnt += 1
g = [[] for _ in range(cnt)]
for key, val in gPyObject.items():
    user1 = transferUser[key]
    for it in val:
        user2 = transferUser[it]
        g[user1].append(user2)
g[:5]

[[1,
  3,
  5,
  9,
  11,
  12,
  14,
  15,
  18,
  22,
  23,
  24,
  26,
  29,
  31,
  35,
  41,
  61,
  67,
  68,
  86,
  88,
  93],
 [0,
  4,
  5,
  9,
  12,
  14,
  15,
  16,
  17,
  18,
  19,
  21,
  22,
  23,
  24,
  26,
  27,
  35,
  42,
  44,
  45,
  46,
  49,
  53,
  54,
  62,
  71,
  73,
  93,
  94,
  95,
  96,
  97,
  104,
  108,
  110,
  117,
  118,
  120,
  123,
  128,
  129,
  130,
  133,
  136,
  139,
  140,
  146,
  155,
  156,
  159,
  175,
  177],
 [6, 85],
 [0,
  9,
  10,
  13,
  18,
  20,
  25,
  29,
  31,
  32,
  33,
  35,
  36,
  38,
  39,
  58,
  63,
  66,
  78,
  81,
  89,
  90,
  98,
  106,
  112,
  115,
  145,
  148],
 [1, 12, 15, 17, 44, 48, 56, 62, 121]]

#### Ghi graph ra file

In [7]:
with open('graph.txt', "w+") as fp:
    fp.write(str(cnt))
    fp.write("\n")
    for i in g:
        line = " ".join([str(item) for item in i])
        fp.write(line)
        fp.write("\n")

    fp.close()

## 3. Cài đặt tuần tự

Các yêu cầu cơ bản của một thuật toán phát hiện cộng đồng nói chung đó là nó phải tự chỉ được ra một phép phân chia tự nhiên giữa các đỉnh mà không đòi hỏi người ta phải xác định số cộng đồng nên có là bao nhiêu, hay giới hạn trong kích thước của các cộng đồng

### 0. Giới thiệu thuật toán Girvan-Newman

- Là thuật toán điển hình nhất trong các thuật toán phân chia đồ thị do nó có  ý nghĩa về mặt lịch sử (đánh dấu bước khởi đầu của một thời kỳ mới trong sự phát triển của phát hiện cộng đồng)
- Đây là một phương pháp chia nhỏ, trong đó các cạnh được loại bỏ dần dần ra khỏi đồ thị
- Các cạnh loại bỏ trong mỗi bước được xác định qua một độ đo trung gian (gọi là betweeness centrality)
- Sau khi loại bỏ mỗi cạnh, độ đo trung gian của các cạnh còn lại sẽ được tính lại dựa vào cạnh bị loại bỏ đó

In [8]:
%%html
<img src='https://drive.google.com/uc?export=view&amp;id=1lRwfBzDtRg0e7D5UIHQ09r84JeDISF-7'/>

Betweeness Centrality - BC (tạm dịch: mức độ tập trung) là một độ đo để thể hiện mức độ ảnh hưởng của một cạnh đối với luồng thông tin trong đồ thị. Nó thường được sử dụng để tìm các cạnh đóng vai trò là cầu nối từ phần này sang phần khác của đồ thị.

Thuật toán tính toán các đường đi ngắn nhất không có trọng số giữa tất cả các cặp nút trong biểu đồ. Mỗi nút nhận được một số điểm, dựa trên số lượng đường đi ngắn nhất đi qua nút. Các cạnh thường xuyên nằm trên đường đi ngắn nhất giữa các cặp nút sẽ có điểm Betweeness Centrality cao hơn. Các cạnh có điểm BC cao nhất sẽ bị loại bỏ sau mỗi lần lặp

In [9]:
%%html
<img src='https://drive.google.com/uc?export=view&amp;id=1u3b_zAQZixUKqDfqrf08Hd3rxPQJV1-y'/>

In [10]:
%%html
<img src='https://drive.google.com/uc?export=view&amp;id=1kv2-5XsuvzbKuCxUkLW31P-AqBWP-DRi'/>

### 1. Thiết kế

Các bước thực hiện:
1. Xác định độ đo betweenness cho các cạnh trong đồ thị bằng thuật toán Girvan Newman
2. Loại bỏ cạnh có betweenness lớn nhất khỏi graph
3. Thực hiện lại bước 2 đến khi không còn cạnh nào

https://colab.research.google.com/drive/1R2Gt2wPEeEL-9_kGfXJRUy24zv1APglX?usp=sharing

### 2. Đánh giá

#### Kịch bản thí nghiệm

In [16]:
f = open("graph.txt", "r")
vertices = int(f.readline())
g = []

for i in range(vertices):
    g.append(list(map(int, f.readline().split())))
f.close()

numEdges = 0
for i in range(len(g)):
  numEdges += len(g[i])
print("Num of edges:", numEdges)

Num of edges: 996


Bộ dữ liệu sử dụng:
- 222 đỉnh
- 996 cạnh (số cạnh tối đa 222 * 221 / 2 = 24531 cạnh)
--> đồ thị thưa

Thực hiện 2 version: \
    + ver1: code python bình thường \
    + ver2: chuyển code về sử dụng mảng numpy và sử dụng decorator @jit(nopython=True)

##### Version 1

In [11]:
%%time
!python sequence.py

['(0, 1) 638.07', '(0, 3) 1400.88', '(0, 5) 170.34', '(0, 9) 387.88', '(0, 11) 189.00', '(0, 12) 155.81', '(0, 14) 81.32', '(0, 15) 129.72', '(0, 18) 100.28', '(0, 22) 90.10', '(0, 23) 51.06', '(0, 24) 53.52', '(0, 26) 152.83', '(0, 29) 177.28', '(0, 31) 79.68', '(0, 35) 22.94', '(0, 41) 189.00', '(0, 61) 189.00', '(0, 67) 561.00', '(0, 68) 119.58', '(0, 86) 98.10', '(0, 88) 189.00', '(0, 93) 106.85', '(1, 4) 390.69', '(1, 5) 105.22', '(1, 9) 2216.62', '(1, 12) 81.85', '(1, 14) 52.12', '(1, 15) 34.76', '(1, 16) 278.00', '(1, 17) 40.70', '(1, 18) 529.75', '(1, 19) 110.46', '(1, 21) 72.52', '(1, 22) 62.95', '(1, 23) 41.95', '(1, 24) 31.55', '(1, 26) 407.32', '(1, 27) 65.88', '(1, 35) 330.57', '(1, 42) 120.98', '(1, 44) 40.75', '(1, 45) 42.84', '(1, 46) 186.50', '(1, 49) 57.35', '(1, 53) 110.46', '(1, 54) 189.00', '(1, 62) 69.58', '(1, 71) 280.00', '(1, 73) 186.50', '(1, 93) 82.15', '(1, 94) 81.95', '(1, 95) 189.00', '(1, 96) 94.76', '(1, 97) 148.21', '(1, 104) 189.00', '(1, 108) 189.00',

##### Version 2

In [12]:
%%time
!python sequenceDecoratorJit.py

['(0, 1) 638.07', '(0, 3) 1400.88', '(0, 5) 170.34', '(0, 9) 387.88', '(0, 11) 189.00', '(0, 12) 155.81', '(0, 14) 81.32', '(0, 15) 129.72', '(0, 18) 100.28', '(0, 22) 90.10', '(0, 23) 51.06', '(0, 24) 53.52', '(0, 26) 152.83', '(0, 29) 177.28', '(0, 31) 79.68', '(0, 35) 22.94', '(0, 41) 189.00', '(0, 61) 189.00', '(0, 67) 561.00', '(0, 68) 119.58', '(0, 86) 98.10', '(0, 88) 189.00', '(0, 93) 106.85', '(1, 4) 390.69', '(1, 5) 105.22', '(1, 9) 2216.62', '(1, 12) 81.85', '(1, 14) 52.12', '(1, 15) 34.76', '(1, 16) 278.00', '(1, 17) 40.70', '(1, 18) 529.75', '(1, 19) 110.46', '(1, 21) 72.52', '(1, 22) 62.95', '(1, 23) 41.95', '(1, 24) 31.55', '(1, 26) 407.32', '(1, 27) 65.88', '(1, 35) 330.57', '(1, 42) 120.98', '(1, 44) 40.75', '(1, 45) 42.84', '(1, 46) 186.50', '(1, 49) 57.35', '(1, 53) 110.46', '(1, 54) 189.00', '(1, 62) 69.58', '(1, 71) 280.00', '(1, 73) 186.50', '(1, 93) 82.15', '(1, 94) 81.95', '(1, 95) 189.00', '(1, 96) 94.76', '(1, 97) 148.21', '(1, 104) 189.00', '(1, 108) 189.00',

#### Kiểm tra độ chính xác


Kiểm tra độ chính xác bằng thư viện NetworkX

In [13]:
%%time
!python testNetworkx.py

['(0, 1) 638.07', '(0, 3) 1400.88', '(0, 5) 170.34', '(0, 9) 387.88', '(0, 11) 189.00', '(0, 12) 155.81', '(0, 14) 81.32', '(0, 15) 129.72', '(0, 18) 100.28', '(0, 22) 90.10', '(0, 23) 51.06', '(0, 24) 53.52', '(0, 26) 152.83', '(0, 29) 177.28', '(0, 31) 79.68', '(0, 35) 22.94', '(0, 41) 189.00', '(0, 61) 189.00', '(0, 67) 561.00', '(0, 68) 119.58', '(0, 86) 98.10', '(0, 88) 189.00', '(0, 93) 106.85', '(1, 4) 390.69', '(1, 5) 105.22', '(1, 9) 2216.62', '(1, 12) 81.85', '(1, 14) 52.12', '(1, 15) 34.76', '(1, 16) 278.00', '(1, 17) 40.70', '(1, 18) 529.75', '(1, 19) 110.46', '(1, 21) 72.52', '(1, 22) 62.95', '(1, 23) 41.95', '(1, 24) 31.55', '(1, 26) 407.32', '(1, 27) 65.88', '(1, 35) 330.57', '(1, 42) 120.98', '(1, 44) 40.75', '(1, 45) 42.84', '(1, 46) 186.50', '(1, 49) 57.35', '(1, 53) 110.46', '(1, 54) 189.00', '(1, 62) 69.58', '(1, 71) 280.00', '(1, 73) 186.50', '(1, 93) 82.15', '(1, 94) 81.95', '(1, 95) 189.00', '(1, 96) 94.76', '(1, 97) 148.21', '(1, 104) 189.00', '(1, 108) 189.00',

## Cài đặt song song

### 1. Thiết kế

### 2. Đánh giá

#### Kịch bản thí nghiệm

Bộ dữ liệu sử dụng:
- 222 đỉnh
- 996 cạnh (số cạnh tối đa 222 * 221 / 2 = 24531 cạnh)
--> đồ thị thưa

#### Chạy code và xem kết quả

In [14]:
%%time
!python parallel.py

(0, 1) 638.07
(0, 3) 1400.88
(0, 5) 170.34
(0, 9) 387.88
(0, 11) 189.00
(0, 12) 155.81
(0, 14) 81.32
(0, 15) 129.72
(0, 18) 100.28
(0, 22) 90.10
(0, 23) 51.06
(0, 24) 53.52
(0, 26) 152.83
(0, 29) 177.28
(0, 31) 79.68
(0, 35) 22.94
(0, 41) 189.00
(0, 61) 189.00
(0, 67) 561.00
(0, 68) 119.58
(0, 86) 98.10
(0, 88) 189.00
(0, 93) 106.85
(1, 0) 638.07
(1, 4) 390.69
(1, 5) 105.22
(1, 9) 2216.62
(1, 12) 81.85
(1, 14) 52.12
(1, 15) 34.76
(1, 16) 278.00
(1, 17) 40.70
(1, 18) 529.75
(1, 19) 110.46
(1, 21) 72.52
(1, 22) 62.95
(1, 23) 41.95
(1, 24) 31.55
(1, 26) 407.32
(1, 27) 65.88
(1, 35) 330.57
(1, 42) 120.98
(1, 44) 40.75
(1, 45) 42.84
(1, 46) 186.50
(1, 49) 57.35
(1, 53) 110.46
(1, 54) 189.00
(1, 62) 69.58
(1, 71) 280.00
(1, 73) 186.50
(1, 93) 82.15
(1, 94) 81.95
(1, 95) 189.00
(1, 96) 94.76
(1, 97) 148.21
(1, 104) 189.00
(1, 108) 189.00
(1, 110) 189.00
(1, 117) 189.00
(1, 118) 189.00
(1, 120) 96.43
(1, 123) 189.00
(1, 128) 189.00
(1, 129) 71.67
(1, 130) 189.00
(1, 133) 110.96
(1, 136) 92.54


#### Kiểm tra độ chính xác